In [1]:
import os
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from model_settings import ms
from convsklearn import asian_trainer

from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
trainer = asian_trainer
root = Path().resolve().parent.parent
models_dir = os.path.join(Path().resolve(),'trained_models','trained_models')
models = [f for f in os.listdir(models_dir) if f.find('ipynb')==-1]
for i,m in enumerate(models):
    print(f"{i}     {m}")

0     .DS_Store
1     2024_10_27 08-05-24 Deep Neural Network asian options


In [2]:
model = models[1]
model_dir = os.path.join(models_dir,model)
model_files = [f for f in os.listdir(model_dir) if f.find('ipynb')==-1 and f.find('.html')==-1]
for i,m in enumerate(model_files):
    print(f"{i}     {m}")

0     2024_10_27 08-05-24 Deep Neural Network asian options insample.csv
1     2024_10_27 08-05-24 Deep Neural Network asian options outsample.csv
2     2024_10_27 08-05-24 Deep Neural Network asian options.pkl
3     2024_10_27 08-05-24 Deep Neural Network asian options.txt


In [3]:
train_data = pd.read_csv(os.path.join(model_dir,[f for f in model_files if f.find('insample')!=-1][0])).iloc[:,1:].copy()
test_data = pd.read_csv(os.path.join(model_dir,[f for f in model_files if f. find('outsample')!=-1][0])).iloc[:,1:].copy()
arrs = trainer.get_train_test_arrays(train_data,test_data)
test_X = arrs['test_X']
test_y = arrs['test_y']
train_X = arrs['train_X']
train_y = arrs['train_y']
n = len(trainer.feature_set)
dnn_params = {
    'alpha': 0.01, 
    'hidden_layer_sizes': (n, n), 
    'learning_rate': 'adaptive', 
    'learning_rate_init': 0.1, 
    'solver': 'sgd'
}

feature_transformers = [
        ('numerical', StandardScaler(), trainer.numerical_features),
        ('categoical', OneHotEncoder(), trainer.categorical_features)
    ]
taget_transformer = StandardScaler()

In [ ]:
preprocessor = ColumnTransformer(
    transformers=feature_transformers
)

dnn_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('dnn', MLPRegressor(**dnn_params))
])


scaled_dnn = TransformedTargetRegressor(
    regressor=dnn_pipeline,
    transformer=taget_transformer
)

dnn_fit = scaled_dnn.fit(train_X,train_y)

In [ ]:
dnn_fit = scaled_dnn.fit(train_X,train_y)

In [ ]:
dnnr=trainer.test_prediction_accuracy(dnn_fit, test_data, train_data)